In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [3]:
os.chdir('/home/azeghost/git/Generative_Models') # adress to git dir
print(os.getcwd())
sep_local = os.sep

/home/azeghost/git/Generative_Models


## Download data prep for Eval_dataset the old method

In [4]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
dataset_name = 'dsprites'
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

In [5]:
from data.gt_load.datasets import load
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.gt_datasets'
absolute = os.path.abspath(DATA_PATH)
print("DATA_PATH",absolute)
# eval_dataset = load(dataset_name='dsprites_full', dataset_path=DATA_PATH)

DATA_PATH /home/azeghost/git/Generative_Models/data/.gt_datasets


In [6]:
def save_to_npz(datasetName: str, X, Y=None, L=None, Z=None):
#     print('Started Saving arrays to npz')
    BASE_PATH = os.path.join(os.getcwd()+os.sep + 'npz_data')
    if not os.path.exists(BASE_PATH):
        os.makedirs(BASE_PATH)
    file_name = "{0}.npz".format(datasetName)
    if os.path.exists(file_name):
        os.remove(file_name)
    np.savez(os.path.join(BASE_PATH, file_name), X=X, Y=Y, L=L, Z=Z)
#     print('Finished Saving arrays to npz')

In [7]:
from tqdm import tqdm
import math

In [8]:
def load_from_npz(BASE_PATH: str, datasetName: str):
#     print('Started loading arrays from npz')
#     BASE_PATH = os.path.join(os.getcwd()+os.sep + 'npz_data')
    file_name = "{0}.npz".format(datasetName)
    data = np.load(os.path.join(BASE_PATH, file_name))
#     print('Finished loading arrays from npz')
    if 'celeb_a' in datasetName:
        return data['X'], data['Y'], data['L']
    return data['X'], data['Y'], data['L'], data['Z']

In [9]:
print(os.getcwd())

/home/azeghost/git/Generative_Models


In [15]:
os.chdir('/home/azeghost/git/Generative_Models')

# Storing Dstripes_full into 10000,64,64 & 10000,6 npz for easy processing 
 - To avoid memory overflow and kernel termination

In [35]:
"""DSprites dataset and new variants with probabilistic decoders."""

import os
from data.gt_load import gt_data, util
import numpy as np
import dask.array as da
import PIL
from six.moves import range
import tensorflow as tf
import json

class DSprites(gt_data.GroundTruthData):
    """DSprites dataset.

    The data set was originally introduced in "beta-VAE: Learning Basic Visual
    Concepts with a Constrained Variational Framework" and can be downloaded from
    https://github.com/deepmind/dsprites-dataset.

    The ground-truth factors of variation are (in the default setting):
    0 - shape (3 different values)
    1 - scale (6 different values)
    2 - orientation (40 different values)
    3 - position x (32 different values)
    4 - position y (32 different values)
    """

    def __init__(self, data_path, scream_path, latents_factor_indices=None):
        # By default, all factors (including shape) are considered ground truth
        # factors.
        self.scream_path = scream_path
        self.data_path = data_path

        if latents_factor_indices is None:
          latents_factor_indices = list(range(6))
        self.latents_factor_indices = latents_factor_indices
        self.data_shape = [64, 64, 1]
        # Load the data so that we can sample from it.
        
        with tf.io.gfile.GFile(self.data_path, "rb") as data_file:
            # Data was saved originally using python2, so we need to set the encoding.
            data = np.load(data_file, encoding="latin1", allow_pickle=True)
            
            
#             for idx, n in tqdm(enumerate(range(0,737280,10000)),  total=math.ceil(737280/10000)):
#                 latents_values = data["latents_values"][n:n+10000]
#                 imgs = data["imgs"][n:n+10000]
#                 latents_names = data["metadata"][()]['latents_names']
#                 latents_classes =  data["latents_classes"][n:n+10000]
                
#                 save_to_npz(datasetName = f'Dstripes_{idx}', X = imgs, Y=latents_values, L = latents_names,
#                            Z = latents_classes)
                
                
#             print(data["imgs"].shape)
#             print(data['latents_values'].shape)
#             print(data["metadata"][()].keys()) #[()]["latents_sizes"]
#             print(data["metadata"][()]['latents_names'])
            
#             self.images = da.from_array(data["imgs"], chunks=100)

#             self.factor_sizes = da.from_array(
#                 data["metadata"][()]["latents_sizes"], chunks=100)

#         print((data["metadata"][()]))

        numpy_transformer.save_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Dstripes_LMDB/meta_info.txt'
                                        , data["metadata"][()])
    
    
#         self.full_factor_sizes = [1, 3, 6, 40, 32, 32]
#         self.factor_bases = np.prod(self.factor_sizes) / np.cumprod(
#             self.factor_sizes)
#         self.state_space = util.SplitDiscreteStateSpace(self.factor_sizes,
#                                                         self.latents_factor_indices)

    @property
    def num_factors(self):
        return self.state_space.num_latents_factors

    @property
    def factors_num_values(self):
        return [self.full_factor_sizes[i] for i in self.latents_factor_indices]

    @property
    def observation_shape(self):
        return self.data_shape

    def sample_factors(self, num, random_state):
        """Sample a batch of factors Y."""
        return self.state_space.sample_latents_factors(num, random_state)

    def sample_observations_from_factors(self, factors, random_state):
        return self.sample_observations_from_factors_no_color(factors, random_state)

    def sample_observations_from_factors_no_color(self, factors, random_state):
        """Sample a batch of observations X given a batch of factors Y."""
        all_factors = self.state_space.sample_all_factors(factors, random_state)
        indices = np.array(np.dot(all_factors, self.factor_bases), dtype=np.int64)
        return np.expand_dims(self.images[indices].astype(np.float32), axis=3)

    def _sample_factor(self, i, num, random_state):
        return random_state.randint(self.factor_sizes[i], size=num)

In [36]:
DSP_PATH = os.path.join(DATA_PATH, 'dsprites', 'dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz')
SCREAM_PATH = os.path.join(DATA_PATH, "scream", "scream.jpg")

DSprites(data_path=DSP_PATH, scream_path=SCREAM_PATH, latents_factor_indices=[1, 2, 3, 4, 5])

In [37]:
numpy_transformer.get_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Dstripes_LMDB/meta_info.txt')

{'date': 'April 2017',
 'description': 'Disentanglement test Sprites dataset.Procedurally generated 2D shapes, from 6 disentangled latent factors.This dataset uses 6 latents, controlling the color, shape, scale, rotation and position of a sprite. All possible variations of the latents are present. Ordering along dimension 1 is fixed and can be mapped back to the exact latent values that generated that image.We made sure that the pixel outputs are different. No noise added.',
 'version': 1,
 'latents_names': ['color', 'shape', 'scale', 'orientation', 'posX', 'posY'],
 'latents_possible_values': {'orientation': [0.0,
   0.16110731556870733,
   0.32221463113741466,
   0.483321946706122,
   0.6444292622748293,
   0.8055365778435367,
   0.966643893412244,
   1.1277512089809514,
   1.2888585245496587,
   1.4499658401183662,
   1.6110731556870734,
   1.7721804712557805,
   1.933287786824488,
   2.0943951023931957,
   2.2555024179619028,
   2.41660973353061,
   2.5777170490993173,
   2.7388243

# Storing from small npz to LMDB

In [9]:
# !rm -rf /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Dstripes_LMDB

In [11]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets')
os.getcwd()
BASE_PATH = os.path.join('/home/azeghost/git/Generative_Models'+os.sep + 'npz_data')

In [12]:
from transformation.lmdb_transformer import LmdbTransformer
from tqdm import tqdm
import math

In [13]:
validation_percentage = 30
valid_format = 'png'

In [14]:
numpy_transformer = LmdbTransformer( validation_percentage, valid_format)

In [14]:
737280

737280

In [15]:
# for idx, n in tqdm(enumerate(range(0,15780,10000)),  total=math.ceil(15780/10000)):
for idx, n in enumerate(range(0,737280,10000)):
#     print(f'Storing {idx} out of {math.ceil(737280/10000)}')
    imgs, latents_values, latents_names, latents_classes  = load_from_npz(BASE_PATH, 
                                                                          datasetName = f'Dstripes_{idx}')

    numpy_transformer.transform_store_from_numpy(images =imgs,
                                                 labels_values =latents_values , 
                                                 labels_names = latents_names,
                                                 labels_classes = latents_classes,
                                                 lmdb_dir='.data/Dstripes_LMDB',
                                                 category='full',
                                                 total_number_imgs = 737280,
                                                 file_idx = idx)
            

  0%|          | 0/10000 [00:00<?, ?it/s]

Storing 0 out of 74


  0%|          | 0/10000 [00:00<?, ?it/s]

Storing 1 out of 74


  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

Storing 72 out of 74


  0%|          | 0/7280 [00:00<?, ?it/s]

Storing 73 out of 74


100%|██████████| 7280/7280 [00:43<00:00, 165.62it/s]


# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [16]:
import lmdb
import pickle

def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]
            
            for label in labels_list:
                if label in labels.keys():
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels.update({label : [eval(f'dataset.{label}') ]})
    
    env.close()
    return {'images': images, **labels}

In [17]:
ds = read_many_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Dstripes_LMDB/_full', 35000)

In [18]:
ds.keys()

dict_keys(['images', 'color_class', 'color_value', 'orientation_class', 'orientation_value', 'posX_class', 'posX_value', 'posY_class', 'posY_value', 'scale_class', 'scale_value', 'shape_class', 'shape_value'])

In [19]:
len(ds['images'])

35000

In [21]:
len(ds['color_value'])

35000

In [38]:
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

# Zip and upload to git

In [39]:
!tar -cvjf dsprites.tar.bz Dstripes_LMDB/*

Dstripes_LMDB/_full/
Dstripes_LMDB/_full/lock.mdb
Dstripes_LMDB/_full/data.mdb
Dstripes_LMDB/meta_info.txt


In [ ]:
!split -b 50M dsprites.tar.bz "dsprites.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [ ]:
#Check if they are created
!ls -lh dsprites.tar.part*

In [ ]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

In [ ]:
!ls -la

In [ ]:
#move to correct folder and push to git
# !mkdir /home/azeghost/git/LMDB_Datasets/pokemon
# !mv pokemon.tar.part* /home/azeghost/git/Generative_Models/data/.pokemon
# !ls -la /home/azeghost/git/Generative_Models/data/.pokemon 

# Combine Them

In [ ]:
#pokemon_training
!cat pokemon.tar.part* > pokemon_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
!ls -la 
!pwd

In [ ]:
!mkdir ./.test

In [ ]:
!tar -xf pokemon_combined.tar.bz --directory ./.test

In [ ]:
!ls -la ./.test/Pokemon_LMDB

In [ ]:
# !rm -rf ./.test

## LMDBImageIterator and LMDBImageGenerator

In [ ]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [ ]:
os.getcwd()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [ ]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

In [ ]:
x, y = training_generator.next()

#  Scale test for Pokemon 

In [ ]:
import os
os.chdir('/home/azeghost/git/LMDB_Datasets')

In [ ]:
#Wrapper class for dataset
class Pokemon_Image:
    def __init__(self, image, label):
        self.channels = image.shape[2]
        self.size = image.shape[:2]
        self.image = image.tobytes()
        self.label = label #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [ ]:
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [ ]:
def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [ ]:
class LMDB_transformer:
    def __init(image_lists**, )
        create_img_list
    
    def transform_store():
        store_lmdb()

In [ ]:
!rm -rf ./LMDB/
!mkdir ./LMDB/
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

# Done on ubuntu system is local machine does not have tar 
 - Install git bash then u can use such commands.
 - Go to windows store and download Ubuntu emulator wich just installs bash
 - On colab these should work